# Predictive Maintenance with Linear Regression Alerts

## Manufacturing Robot Current Monitoring - Failure Prediction System

This notebook implements a Predictive Maintenance Alert System using Linear Regression models to detect anomalies and predict failures in industrial robot current data.

### Workflow:
1. **Database Integration** - Connect to Neon.tech PostgreSQL, ingest training data
2. **Model Training** - Fit univariate linear regression for each axis (Time → Current)
3. **Residual Analysis** - Analyze prediction errors to discover thresholds
4. **Alert System** - Implement ALERT/ERROR detection based on sustained deviations
5. **Testing** - Process synthetic failure data and log events
6. **Visualization** - Generate alert dashboard

## Cell 1: Imports and Setup

In [1]:
import sys
import os
import importlib

# Add src directory to Python path
project_root = os.path.dirname(os.getcwd()) if os.path.basename(os.getcwd()) == 'notebook' else os.getcwd()
src_dir = os.path.join(project_root, 'src')
if src_dir not in sys.path:
    sys.path.insert(0, src_dir)

print(f"Project root: {project_root}")
print(f"Source directory: {src_dir}")

# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Use Qt backend for separate windows
%matplotlib qt

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Import custom modules (force reload to pick up any source changes)
import database_utils
import linear_regression_model
import alert_system
importlib.reload(database_utils)
importlib.reload(linear_regression_model)
importlib.reload(alert_system)

from database_utils import (connect_to_db, create_training_table, 
                            ingest_training_data, query_training_data,
                            get_training_record_count, create_alerts_table,
                            clear_training_table, drop_training_table)
from linear_regression_model import RobotRegressionModels
from alert_system import AlertSystem, AlertThresholds

print("\nLibraries imported successfully")
print(f"Matplotlib backend: {plt.get_backend()}")

Project root: c:\Projects\Lab1_StreamingDataforPMwithLinRegAlerts
Source directory: c:\Projects\Lab1_StreamingDataforPMwithLinRegAlerts\src

Libraries imported successfully
Matplotlib backend: qtagg


## Cell 2: Database Integration

Connect to Neon.tech PostgreSQL and ingest training data.

In [2]:
# Database configuration (Neon.tech PostgreSQL)
db_config = {
    'host': 'ep-polished-snow-ahx3qiod-pooler.c-3.us-east-1.aws.neon.tech',
    'database': 'neondb',
    'user': 'neondb_owner',
    'password': 'npg_JlIENr3i4AbL',
    'port': 5432,
    'sslmode': 'require'
}

# Training data path
training_csv_path = os.path.join(project_root, 'data', 'robots_combined_traindata.csv')

print("=" * 70)
print("DATABASE INTEGRATION")
print("=" * 70)

# Connect to database
conn = connect_to_db(db_config)

# Create tables
create_training_table(conn)
create_alerts_table(conn)

# Check existing records
existing_count = get_training_record_count(conn)
print(f"\nExisting training records in database: {existing_count:,}")

DATABASE INTEGRATION
Connected to database: neondb @ ep-polished-snow-ahx3qiod-pooler.c-3.us-east-1.aws.neon.tech
Training data table created/verified
Alerts table created/verified

Existing training records in database: 40,000


In [ ]:
# Ingest training data (only if table is empty or needs refresh)
FORCE_REINGEST = True  # Set to True to reload data after regenerating CSVs

if existing_count == 0 or FORCE_REINGEST:
    if FORCE_REINGEST:
        print("Dropping and recreating training table (schema refresh)...")
        drop_training_table(conn)
        create_training_table(conn)
    
    print(f"\nIngesting training data from: {training_csv_path}")
    records_ingested = ingest_training_data(conn, training_csv_path)
    print(f"Total records ingested: {records_ingested:,}")
else:
    print(f"Using existing {existing_count:,} records in database")
    print("(Set FORCE_REINGEST=True to reload data)")

In [3]:
# Robot configuration: name -> number of active axes
ROBOT_CONFIG = {
    'Robot A': 8,
    'Robot B': 10,
    'Robot C': 12,
    'Robot D': 12,
}

# Query training data per robot from database
print("\nQuerying training data per robot from database...")
print("=" * 70)

training_data = {}  # robot_name -> DataFrame

for robot_name, n_axes in ROBOT_CONFIG.items():
    df_robot = query_training_data(conn, robot_name=robot_name)
    # Drop columns that are entirely NaN (axes this robot doesn't have)
    df_robot = df_robot.dropna(axis=1, how='all')
    training_data[robot_name] = df_robot

    active_axes = [c for c in df_robot.columns if c.startswith('axis_')]
    print(f"\n{robot_name}: {df_robot.shape[0]:,} records, "
          f"{len(active_axes)} active axes {active_axes}")

print("\n" + "=" * 70)
print(f"Total robots loaded: {len(training_data)}")


Querying training data per robot from database...
Retrieved 10,000 records from database

Robot A: 10,000 records, 8 active axes ['axis_1', 'axis_2', 'axis_3', 'axis_4', 'axis_5', 'axis_6', 'axis_7', 'axis_8']
Retrieved 10,000 records from database

Robot B: 10,000 records, 10 active axes ['axis_1', 'axis_2', 'axis_3', 'axis_4', 'axis_5', 'axis_6', 'axis_7', 'axis_8', 'axis_9', 'axis_10']
Retrieved 10,000 records from database

Robot C: 10,000 records, 12 active axes ['axis_1', 'axis_2', 'axis_3', 'axis_4', 'axis_5', 'axis_6', 'axis_7', 'axis_8', 'axis_9', 'axis_10', 'axis_11', 'axis_12']
Retrieved 10,000 records from database

Robot D: 10,000 records, 12 active axes ['axis_1', 'axis_2', 'axis_3', 'axis_4', 'axis_5', 'axis_6', 'axis_7', 'axis_8', 'axis_9', 'axis_10', 'axis_11', 'axis_12']

Total robots loaded: 4


## Cell 3: Model Training

Fit univariate linear regression per robot, per axis:
- **Model**: y = slope × time_index + intercept
- **Each robot** gets its own `RobotRegressionModels` instance
- Only axes with data are fitted (NaN-only axes are skipped automatically)

In [4]:
# Train per-robot regression models
robot_models = {}  # robot_name -> RobotRegressionModels
all_summaries = []

for robot_name, df_robot in training_data.items():
    print(f"\n{'#' * 70}")
    print(f"# {robot_name}")
    print(f"{'#' * 70}")

    models = RobotRegressionModels()
    models.train_all_axes(df_robot)
    robot_models[robot_name] = models

    # Collect summary with robot column
    summary = models.get_model_summary()
    summary.insert(0, 'robot', robot_name)
    all_summaries.append(summary)

# Combined model summary
df_all_params = pd.concat(all_summaries, ignore_index=True)

print("\n" + "=" * 70)
print("COMBINED MODEL PARAMETERS SUMMARY")
print("=" * 70)
print(df_all_params[df_all_params['is_fitted'] == True].to_string())

# Save combined model parameters
params_path = os.path.join(project_root, 'data', 'model_params.csv')
df_all_params.to_csv(params_path, index=False)
print(f"\nModel parameters saved to: {params_path}")

# Quick count
for robot_name, models in robot_models.items():
    fitted = len([m for m in models.models.values() if m.is_fitted])
    print(f"  {robot_name}: {fitted} fitted models")


######################################################################
# Robot A
######################################################################
TRAINING LINEAR REGRESSION MODELS

Training model for axis_1...
  Slope: 0.000002
  Intercept: -0.006324
  Residual Std: 0.999947

Training model for axis_2...
  Slope: -0.000000
  Intercept: -0.040068
  Residual Std: 1.002516

Training model for axis_3...
  Slope: -0.000000
  Intercept: 0.012375
  Residual Std: 0.995143

Training model for axis_4...
  Slope: -0.000004
  Intercept: 0.032395
  Residual Std: 1.013031

Training model for axis_5...
  Slope: 0.000005
  Intercept: 0.024623
  Residual Std: 1.006569

Training model for axis_6...
  Slope: 0.000006
  Intercept: -0.017623
  Residual Std: 0.993321

Training model for axis_7...
  Slope: -0.000002
  Intercept: 0.001587
  Residual Std: 1.002879

Training model for axis_8...
  Slope: 0.000003
  Intercept: 0.028459
  Residual Std: 0.991359

TRAINING COMPLETE

##########################

## Cell 4: Regression Visualization

Scatter plots with regression lines per robot.

In [5]:
# Plot regression lines per robot
print("Generating per-robot regression plots...")

for robot_name, models in robot_models.items():
    safe_name = robot_name.replace(' ', '_')
    save_path = os.path.join(project_root, 'alerts', f'regression_lines_{safe_name}.png')
    fig = models.plot_regression_lines(
        training_data[robot_name],
        title_prefix=f"{robot_name} Training: ",
        save_path=save_path
    )
    plt.show()

Generating per-robot regression plots...
Regression plot saved to: c:\Projects\Lab1_StreamingDataforPMwithLinRegAlerts\alerts\regression_lines_Robot_A.png
Regression plot saved to: c:\Projects\Lab1_StreamingDataforPMwithLinRegAlerts\alerts\regression_lines_Robot_B.png
Regression plot saved to: c:\Projects\Lab1_StreamingDataforPMwithLinRegAlerts\alerts\regression_lines_Robot_C.png
Regression plot saved to: c:\Projects\Lab1_StreamingDataforPMwithLinRegAlerts\alerts\regression_lines_Robot_D.png


## Cell 5: Residual Analysis

Analyze residuals (observed - predicted) per robot to:
1. Understand prediction error distribution
2. Identify outlier patterns
3. Determine appropriate thresholds for alerts

In [6]:
# Plot residual distributions per robot
print("Generating per-robot residual histograms...")

for robot_name, models in robot_models.items():
    safe_name = robot_name.replace(' ', '_')
    save_path = os.path.join(project_root, 'alerts', f'residual_histograms_{safe_name}.png')
    fig = models.plot_residual_analysis(
        training_data[robot_name],
        save_path=save_path
    )
    fig.suptitle(f'{robot_name} - Residual Distributions', fontsize=16, fontweight='bold')
    plt.show()

Generating per-robot residual histograms...
Residual analysis plot saved to: c:\Projects\Lab1_StreamingDataforPMwithLinRegAlerts\alerts\residual_histograms_Robot_A.png
Residual analysis plot saved to: c:\Projects\Lab1_StreamingDataforPMwithLinRegAlerts\alerts\residual_histograms_Robot_B.png
Residual analysis plot saved to: c:\Projects\Lab1_StreamingDataforPMwithLinRegAlerts\alerts\residual_histograms_Robot_C.png
Residual analysis plot saved to: c:\Projects\Lab1_StreamingDataforPMwithLinRegAlerts\alerts\residual_histograms_Robot_D.png


In [7]:
# Residual boxplots per robot
print("Generating per-robot residual boxplots...")

for robot_name, models in robot_models.items():
    safe_name = robot_name.replace(' ', '_')
    save_path = os.path.join(project_root, 'alerts', f'residual_boxplots_{safe_name}.png')
    fig = models.plot_residual_boxplots(
        training_data[robot_name],
        save_path=save_path
    )
    fig.suptitle(f'{robot_name} - Residual Boxplots', fontsize=16, fontweight='bold')
    plt.show()

Generating per-robot residual boxplots...
Residual boxplot saved to: c:\Projects\Lab1_StreamingDataforPMwithLinRegAlerts\alerts\residual_boxplots_Robot_A.png
Residual boxplot saved to: c:\Projects\Lab1_StreamingDataforPMwithLinRegAlerts\alerts\residual_boxplots_Robot_B.png
Residual boxplot saved to: c:\Projects\Lab1_StreamingDataforPMwithLinRegAlerts\alerts\residual_boxplots_Robot_C.png
Residual boxplot saved to: c:\Projects\Lab1_StreamingDataforPMwithLinRegAlerts\alerts\residual_boxplots_Robot_D.png


In [8]:
# Compute detailed residual statistics per robot
print("=" * 70)
print("RESIDUAL STATISTICS (Per Robot)")
print("=" * 70)

all_residual_stats = []

for robot_name, models in robot_models.items():
    df_robot = training_data[robot_name]
    time_index = np.arange(len(df_robot))

    for axis_name in models.AXIS_NAMES:
        if axis_name in df_robot.columns and models.models[axis_name].is_fitted:
            residuals = models.models[axis_name].get_residuals(time_index, df_robot[axis_name])
            residuals = residuals[~np.isnan(residuals)]

            if len(residuals) > 0:
                stats = {
                    'robot': robot_name,
                    'axis': axis_name,
                    'mean': np.mean(residuals),
                    'std': np.std(residuals),
                    'min': np.min(residuals),
                    'max': np.max(residuals),
                    'q1': np.percentile(residuals, 25),
                    'median': np.median(residuals),
                    'q3': np.percentile(residuals, 75),
                    'p95': np.percentile(residuals, 95),
                    'p99': np.percentile(residuals, 99)
                }
                all_residual_stats.append(stats)

if all_residual_stats:
    df_residual_stats = pd.DataFrame(all_residual_stats).round(4)
    print(df_residual_stats.to_string())

RESIDUAL STATISTICS (Per Robot)
      robot     axis  mean     std     min     max      q1  median      q3     p95     p99
0   Robot A   axis_1   0.0  0.9999 -0.7995  4.3486 -0.5962 -0.4789  0.1882  2.2167  3.7139
1   Robot A   axis_2   0.0  1.0025 -1.5610  3.3622 -0.8508  0.2035  0.3669  1.9853  2.9653
2   Robot A   axis_3   0.0  0.9951 -1.5627  3.6589 -0.6864 -0.0381  0.2143  2.1260  3.2587
3   Robot A   axis_4  -0.0  1.0130 -1.0526  4.2861 -0.7273 -0.3783  0.4890  2.2423  3.7069
4   Robot A   axis_5  -0.0  1.0066 -1.1815  3.6395 -0.7742 -0.3833  0.8308  1.8452  2.9195
5   Robot A   axis_6  -0.0  0.9933 -0.8282  4.7201 -0.5878 -0.2983  0.0656  2.6201  3.8978
6   Robot A   axis_7   0.0  1.0029 -0.9212  1.7442 -0.7651 -0.5835  1.4513  1.6813  1.7021
7   Robot A   axis_8   0.0  0.9914 -0.6588  6.1979 -0.3324 -0.2496 -0.1338  3.2554  4.4135
8   Robot B   axis_1  -0.0  0.9970 -0.7871  4.3422 -0.6024 -0.4818  0.2258  2.2261  3.6704
9   Robot B   axis_2   0.0  1.0063 -1.5476  3.3800 -0.8493

## Cell 6: Threshold Discovery & Justification

Based on residual analysis, define alert thresholds:

- **MinC**: Minimum deviation for ALERT (scaled by residual_std)
- **MaxC**: Maximum deviation for ERROR (scaled by residual_std)
- **T**: Minimum sustained duration in seconds

### Justification:

1. **MinC = 2.0**: Values exceeding 2σ from the regression line represent approximately 5% of normal variance (beyond 95% confidence interval). In predictive maintenance, this indicates early signs of degradation.

2. **MaxC = 3.0**: Values exceeding 3σ represent less than 0.3% of normal variance (beyond 99.7% confidence interval). This level indicates significant deviation requiring immediate attention.

3. **T = 30 seconds**: Transient spikes are common during robot operation cycles. A 30-second sustained deviation filters out momentary fluctuations while capturing genuine degradation patterns.

In [9]:
# Define thresholds based on residual analysis
print("=" * 70)
print("THRESHOLD CONFIGURATION")
print("=" * 70)

# Thresholds (multipliers of residual standard deviation)
MinC = 2.0   # Alert threshold: 2σ above regression
MaxC = 3.0   # Error threshold: 3σ above regression  
T = 30       # Sustained duration: 30 seconds

thresholds = AlertThresholds(MinC=MinC, MaxC=MaxC, T=T)
print(f"\nAlert Thresholds: {thresholds}")

print(f"\n--- THRESHOLD JUSTIFICATION ---")
print(f"""\nMinC = {MinC} (Alert Threshold)
  - Values exceeding {MinC}σ from regression line
  - Represents ~{100 - 95.45:.1f}% of normal variance (beyond 95% CI)
  - Indicates: Early signs of potential degradation
  - Action: Schedule inspection, monitor closely
""")

print(f"""MaxC = {MaxC} (Error Threshold)
  - Values exceeding {MaxC}σ from regression line
  - Represents ~{100 - 99.73:.2f}% of normal variance (beyond 99.7% CI)
  - Indicates: Significant anomaly, potential failure imminent
  - Action: Immediate attention required
""")

print(f"""T = {T} seconds (Sustained Duration)
  - Deviation must persist for {T} consecutive seconds
  - Filters out momentary spikes from normal operation
  - Based on typical robot operational cycle patterns
  - Action: Only alert on persistent issues
""")

# Show actual threshold values per robot per axis
print("\nActual threshold values per robot per axis (in normalized units):")
for robot_name, models in robot_models.items():
    print(f"\n  {robot_name}:")
    for axis_name in models.AXIS_NAMES:
        if models.models[axis_name].is_fitted:
            std = models.models[axis_name].residual_std
            print(f"    {axis_name}: ALERT >= {MinC * std:.4f}, ERROR >= {MaxC * std:.4f}")
        else:
            print(f"    {axis_name}: No model (no data)")

THRESHOLD CONFIGURATION

Alert Thresholds: AlertThresholds(MinC=2.0, MaxC=3.0, T=30s)

--- THRESHOLD JUSTIFICATION ---

MinC = 2.0 (Alert Threshold)
  - Values exceeding 2.0σ from regression line
  - Represents ~4.5% of normal variance (beyond 95% CI)
  - Indicates: Early signs of potential degradation
  - Action: Schedule inspection, monitor closely

MaxC = 3.0 (Error Threshold)
  - Values exceeding 3.0σ from regression line
  - Represents ~0.27% of normal variance (beyond 99.7% CI)
  - Indicates: Significant anomaly, potential failure imminent
  - Action: Immediate attention required

T = 30 seconds (Sustained Duration)
  - Deviation must persist for 30 consecutive seconds
  - Filters out momentary spikes from normal operation
  - Based on typical robot operational cycle patterns
  - Action: Only alert on persistent issues


Actual threshold values per robot per axis (in normalized units):

  Robot A:
    axis_1: ALERT >= 1.9999, ERROR >= 2.9998
    axis_2: ALERT >= 2.0050, ERROR >= 

## Cell 7: Alert System Setup

In [10]:
# Initialize per-robot alert systems
robot_alert_systems = {}  # robot_name -> AlertSystem

for robot_name, models in robot_models.items():
    robot_alert_systems[robot_name] = AlertSystem(models, thresholds)

print("Per-robot Alert Systems initialized")
print(f"Thresholds: {thresholds}")
for robot_name, asys in robot_alert_systems.items():
    fitted = len([m for m in asys.models.models.values() if m.is_fitted])
    print(f"  {robot_name}: monitoring {fitted} axes")

# Create output directories
logs_dir = os.path.join(project_root, 'logs')
alerts_dir = os.path.join(project_root, 'alerts')
os.makedirs(logs_dir, exist_ok=True)
os.makedirs(alerts_dir, exist_ok=True)

print(f"\nLogs directory: {logs_dir}")
print(f"Alerts directory: {alerts_dir}")

Per-robot Alert Systems initialized
Thresholds: AlertThresholds(MinC=2.0, MaxC=3.0, T=30s)
  Robot A: monitoring 8 axes
  Robot B: monitoring 10 axes
  Robot C: monitoring 12 axes
  Robot D: monitoring 12 axes

Logs directory: c:\Projects\Lab1_StreamingDataforPMwithLinRegAlerts\logs
Alerts directory: c:\Projects\Lab1_StreamingDataforPMwithLinRegAlerts\alerts


## Cell 8: Streaming Test Simulation

Process the synthetic test data (`robots_combined_v2.csv`) which contains injected failure behavior.

In [11]:
# Load test data (with injected failures)
test_csv_path = os.path.join(project_root, 'data', 'robots_combined_v2.csv')
print(f"Loading test data from: {test_csv_path}")

df_test_all = pd.read_csv(test_csv_path)
print(f"\nTest Data Shape: {df_test_all.shape}")
print(f"Columns: {list(df_test_all.columns)}")
print(f"Robots in test data: {df_test_all['robot'].unique()}")
print(f"\nRecords per robot:")
print(df_test_all['robot'].value_counts().to_string())

Loading test data from: c:\Projects\Lab1_StreamingDataforPMwithLinRegAlerts\data\robots_combined_v2.csv

Test Data Shape: (200000, 14)
Columns: ['timestamp', 'robot', 'axis_1', 'axis_2', 'axis_3', 'axis_4', 'axis_5', 'axis_6', 'axis_7', 'axis_8', 'axis_9', 'axis_10', 'axis_11', 'axis_12']
Robots in test data: <StringArray>
['Robot A', 'Robot B', 'Robot C', 'Robot D']
Length: 4, dtype: str

Records per robot:
robot
Robot A    50000
Robot B    50000
Robot C    50000
Robot D    50000


In [12]:
# Process test data through per-robot alert systems
print("Processing test data per robot...")
print("=" * 70)

for robot_name, asys in robot_alert_systems.items():
    df_robot_test = df_test_all[df_test_all['robot'] == robot_name].reset_index(drop=True)
    # Drop NaN-only axis columns for this robot
    df_robot_test = df_robot_test.dropna(axis=1, how='all')

    print(f"\n{robot_name}: {len(df_robot_test):,} test records")
    asys.process_streaming_data(df_robot_test, time_interval_seconds=1)

Processing test data per robot...

Robot A: 50,000 test records
PROCESSING STREAMING DATA FOR ALERTS
Thresholds: AlertThresholds(MinC=2.0, MaxC=3.0, T=30s)
Processing 50,000 records...
  Processed 10,000 / 50,000 records, 0 events logged
  Processed 20,000 / 50,000 records, 0 events logged
  Processed 30,000 / 50,000 records, 0 events logged
  Processed 40,000 / 50,000 records, 0 events logged
  Processed 50,000 / 50,000 records, 82 events logged

Processing complete: 82 total events logged

Robot B: 50,000 test records
PROCESSING STREAMING DATA FOR ALERTS
Thresholds: AlertThresholds(MinC=2.0, MaxC=3.0, T=30s)
Processing 50,000 records...
  Processed 10,000 / 50,000 records, 0 events logged
  Processed 20,000 / 50,000 records, 0 events logged
  Processed 30,000 / 50,000 records, 0 events logged
  Processed 40,000 / 50,000 records, 0 events logged
  Processed 50,000 / 50,000 records, 543 events logged

Processing complete: 543 total events logged

Robot C: 50,000 test records
PROCESSING

In [ ]:
# Display combined alert summary
print("=" * 70)
print("ALERT DETECTION RESULTS (All Robots)")
print("=" * 70)

# Merge all alert logs
all_events = []
for robot_name, asys in robot_alert_systems.items():
    alert_count = len([e for e in asys.alert_log if e.event_type == 'ALERT'])
    error_count = len([e for e in asys.alert_log if e.event_type == 'ERROR'])
    print(f"\n{robot_name}: {len(asys.alert_log)} events "
          f"(ALERTs: {alert_count}, ERRORs: {error_count})")
    all_events.extend(asys.alert_log)

print(f"\nTotal events across all robots: {len(all_events)}")

if all_events:
    df_events = pd.DataFrame([e.to_dict() for e in all_events])
    print("\nSummary by robot, axis, event_type:")
    summary = df_events.groupby(['robot', 'axis', 'event_type']).agg(
        count=('deviation', 'size'),
        mean_deviation=('deviation', 'mean'),
        max_deviation=('deviation', 'max'),
        mean_duration=('duration_seconds', 'mean'),
    ).round(3)
    print(summary.to_string())

    print("\nSample Events:")
    for event in all_events[:5]:
        print(f"  {event}")
else:
    print("\nNo alerts detected in test data.")

## Cell 9: Event Logging

In [ ]:
# Save combined alert log to CSV
log_path = os.path.join(logs_dir, 'alert_log.csv')

# Combine all events from all robot alert systems
all_events = []
for robot_name, asys in robot_alert_systems.items():
    all_events.extend(asys.alert_log)

# Write combined log
if all_events:
    df_log = pd.DataFrame([e.to_dict() for e in all_events])
    df_log.to_csv(log_path, index=False)
    print(f"Combined alert log saved to: {log_path} ({len(df_log)} events)")
    print(f"\nLog file preview:")
    print(df_log.head(10).to_string())
else:
    print("No events to save.")

## Cell 10: Alert Dashboard Visualization

Generate comprehensive dashboard showing all 12 axes with alert/error markers.

In [ ]:
# Generate per-robot alert dashboards
print("Generating per-robot alert dashboards...")

for robot_name, asys in robot_alert_systems.items():
    safe_name = robot_name.replace(' ', '_')
    dashboard_path = os.path.join(alerts_dir, f'alert_dashboard_{safe_name}.png')

    # Get this robot's test data
    df_robot_test = df_test_all[df_test_all['robot'] == robot_name].reset_index(drop=True)
    df_robot_test = df_robot_test.dropna(axis=1, how='all')

    fig = asys.generate_alert_dashboard(df_robot_test, save_path=dashboard_path)
    fig.suptitle(f'{robot_name} - Alert Dashboard', fontsize=18, fontweight='bold')
    plt.show()

## Cell 11: Final Summary

In [ ]:
# Final summary
print("=" * 70)
print("PREDICTIVE MAINTENANCE ANALYSIS COMPLETE")
print("=" * 70)

total_fitted = 0
total_alerts = 0
total_errors = 0

print("\nPER-ROBOT BREAKDOWN:")
print("-" * 70)

for robot_name in ROBOT_CONFIG:
    models = robot_models[robot_name]
    asys = robot_alert_systems[robot_name]
    df_train = training_data[robot_name]
    df_robot_test = df_test_all[df_test_all['robot'] == robot_name]

    fitted = len([m for m in models.models.values() if m.is_fitted])
    alerts = len([e for e in asys.alert_log if e.event_type == 'ALERT'])
    errors = len([e for e in asys.alert_log if e.event_type == 'ERROR'])

    total_fitted += fitted
    total_alerts += alerts
    total_errors += errors

    print(f"\n  {robot_name}:")
    print(f"    Fitted models:  {fitted}")
    print(f"    Training rows:  {len(df_train):,}")
    print(f"    Test rows:      {len(df_robot_test):,}")
    print(f"    ALERTs:         {alerts}")
    print(f"    ERRORs:         {errors}")

print(f"""
{'=' * 70}
COMBINED TOTALS:
  Total fitted models: {total_fitted}
  Total training rows: {sum(len(d) for d in training_data.values()):,}
  Total test rows:     {len(df_test_all):,}
  Total ALERTs:        {total_alerts}
  Total ERRORs:        {total_errors}

THRESHOLDS:
  MinC (Alert): {thresholds.MinC}σ above regression
  MaxC (Error): {thresholds.MaxC}σ above regression
  T (Duration): {thresholds.T} seconds sustained

OUTPUT FILES:
  Model Parameters:     data/model_params.csv (combined, with robot column)
  Alert Log:            logs/alert_log.csv (combined, all robots)
  Regression Plots:     alerts/regression_lines_Robot_*.png (x{len(ROBOT_CONFIG)})
  Residual Histograms:  alerts/residual_histograms_Robot_*.png (x{len(ROBOT_CONFIG)})
  Residual Boxplots:    alerts/residual_boxplots_Robot_*.png (x{len(ROBOT_CONFIG)})
  Alert Dashboards:     alerts/alert_dashboard_Robot_*.png (x{len(ROBOT_CONFIG)})
{'=' * 70}
""")

# Close database connection
conn.close()
print("Database connection closed.")